In [1]:
import re
import math
import sys

import numpy
import pandas

import ipywidgets as widgets
from IPython.display import display, clear_output

import gensim
from gensim.corpora import Dictionary
from gensim.models import TfidfModel

import pythainlp

from sklearn.cluster import KMeans

from data_tokenizer import load_corpus

### Load Data

In [21]:
file_name = 'ผู้บริโภค - TrueCoffee.txt'

corpus, labels = load_corpus('../data/facebook/' + file_name)

len_corpus = len(corpus)
print('Total documents', len_corpus)

clusters = list(set(labels))
print(len(clusters), 'clusters')

f = open('../data/facebook/tokenized/tokenized_' + file_name)
tokenized_corpus = eval(f.read())
f.close()

Total documents 353
1 clusters


### Preprocess Corpus

#### Remove Words

In [22]:
dictionary = Dictionary(tokenized_corpus)
print('origin:', len(dictionary), 'words')

dictionary.filter_extremes(no_below=2, no_above=0.7, keep_n=len(dictionary))
print('filter frequent words:', len(dictionary), 'words')

letter_words = [id for id in range(len(dictionary)) if len(dictionary[id]) <= 1] 
dictionary.filter_tokens(bad_ids=letter_words)
print('filter letter words:', len(dictionary), 'words')

stopwords = pythainlp.corpus.stopwords.words('thai')
stopwords.append('นี้')
dictionary.add_documents([stopwords])
stopwords = [dictionary.token2id[word] for word in stopwords]
dictionary.filter_tokens(bad_ids=stopwords)
print('filter stop words:', len(dictionary), 'words')

origin: 953 words
filter frequent words: 374 words
filter letter words: 372 words
filter stop words: 221 words


In [23]:
# bow_corpus = [dictionary.doc2bow(doc) for doc in tokenized_corpus]
idx_corpus = [dictionary.doc2idx(doc) for doc in tokenized_corpus]

temp_corpus = []
for doc in idx_corpus:
    temp_corpus.append([dictionary[id] for id in doc if id >= 0])
idx_corpus = temp_corpus

#### Dimension Reduction

In [24]:
average_doc_size = 0
for doc in idx_corpus:
    average_doc_size += len(doc)
average_doc_size /= len(idx_corpus)
average_doc_size = math.ceil(average_doc_size)
average_doc_size

df = dictionary.dfs
filtered_corpus = []
for doc in idx_corpus:
    new_doc = [(word, df[dictionary.token2id[word]]) for word in doc]
    new_doc.sort(reverse=True, key=lambda x: x[1])
    new_doc = new_doc[:average_doc_size]
    filtered_corpus.append([word for word, df in new_doc])

### K-Mean

In [25]:
def get_bow(corpus):
    new_dict = Dictionary(corpus)

    # new_dict.filter_extremes(no_below=2, no_above=1, keep_n=len(new_dict))
    # print(len(new_dict))

    unique_words = [new_dict[id] for id in range(len(new_dict))]
    array = numpy.zeros((len_corpus, len(unique_words)), dtype=float)
    
    for i, doc in enumerate(corpus):
        for word in doc:
            array[i, new_dict.token2id[word]] += 1

        ## normalization
        if len(doc) != 0:
            array[i] = numpy.divide(array[i], len(doc))

    return pandas.DataFrame(array, columns=unique_words, dtype=float)

In [26]:
def predict_cluster(bow_corpus, num_clusters):
    kmeans = KMeans(n_clusters=num_clusters).fit(bow_corpus)
    predicted_labels = kmeans.labels_

    result = pandas.DataFrame()
    result['comment'] = corpus
    result['tokenized_comment'] = idx_corpus
    result['label'] = labels
    result['predicted_label'] = predicted_labels
    
    return result

In [27]:
num_clusters = 6
# bow_corpus = get_bow(idx_corpus)
bow_corpus = get_bow(filtered_corpus)
result = predict_cluster(bow_corpus, num_clusters)
bow_corpus.head()

,กาแฟ,ชง,ร้าน,สวย,ทรูมูฟ,ลูกค้า,สิทธิ์,เต็ม,โปร,รสชาติ,...,แถว,ชัด,บริการ,โปรโมชั่น,ชม,เวลา,ชาไทย,หวัง,เข้มข้น,ดืม
0,0.2,0.2,0.2,0.4,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.2,0.2,0.2,0.2,0.2,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.333333,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.4,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Result

In [28]:
label_count = numpy.unique(result['predicted_label'], return_counts=True) 
print(label_count)

for cluster in clusters:
    print('\t' + cluster, end='')
print('\tpercent')

for label in range(len(clusters)):
    print(str(label) + '  |', end='')
    
    num_max = 0
    for cluster in clusters:
        loc = result[(result['label'] == cluster) & (result['predicted_label'] == label)]
        if len(loc) > num_max:
            num_max = len(loc)
        print('\t' + str(len(loc)), end='')
    
    print('\t' + str(num_max / label_count[1][label]))

(array([0, 1, 2, 3, 4, 5], dtype=int32), array([ 28, 222,  22,  14,  17,  50]))
		percent
0  |	28	1.0


In [29]:
comment_widget = widgets.ToggleButtons(
    options=[num + 1 for num in range(num_clusters + 1)],
    disabled=False,
    button_style='',
)

def on_comment_widget_click(change):
    clear_output()
    display(comment_widget)
    print('cluster', change['new'])
    for index, value in result[result['predicted_label'] == change['new']]['comment'].iteritems():
        print(index, value)

comment_widget.observe(on_comment_widget_click, names='index')
on_comment_widget_click({'new' : 0})

ToggleButtons(options=(1, 2, 3, 4, 5, 6), value=1)

18 นี้จะเหมือนอเมซอน แต่ละสาขารสชาติไม่เหมือนกัน คนทำกาแฟเปลี่ยนรสชาติก็เปลี่ยน ปลเมิงควรให้ลูกค้าที่ใช้ทรูโดยเฉพาะได้สิทธิ์ได้ส่วนลดพิเศษบ้าง
20 ชอคโกแลตปั่นรสชาติเหมือนเอาแต่ละแก้วที่ไม่หมดมาผสมรวมกันให้   ชาเขียวน้ำแข็งไม่มีเลย  เละเป็นโจ๊ก ไม่เย็นแพงเกินคุณภาพจริงๆ
28 ซื้อเพราะหาที่นั่งล้วนๆ รสชาติไม่ต้องพูดถึง ปล่อยน้ำแข็งละลายทุกที5555
42 เน๊ตบ้านนิ่งมากเหมือนรสชาติดูดแล้วไม่รู้อารมณ์จะไปทางไหน จะขมก็ไม่ใช่ จะหวานก็ไม่มีเติมน้ำเชื่อมก็แล้ว ที่สำคัญกินเสร็จขี้แตกบาย
66 สั่งคาปูชิโน่เย็น แต่สิ่งที่ได้ นึกว่านมสดเย็น ไม่มีรสชาติกาแฟเลย ควรไปเรียนนะ
84 สาขาเซ็นทรัลพัทยา รสชาติแย่มาก จืด แก้วเล็ก 100฿ ไม่โอเคเลยคะ
103 รสชาติ ส้วนตินมาก 5555
130 รสชาติไม่อร่อยไอ้ที่มีวิปครีม ท็อปปิ้งต่างๆก็หวานเจี๊ยบ
146 รสชาติกาแฟค่ะ ไม่ได้เรื่อง ขายแต่ซิมกะโทสับต่อไปเถอะ
147 รสชาติกาแฟค่ะ อ่อนเกินไป จืดด้วย ไม่อร่อยเลย
167 รสชาติงั้นๆ
168 จริง ไม่เคยใช้สิทธิ์ได้เลย รสชาติงั้นๆ
196 รสชาติไม่อร่อยเลยค่ะ
219 รสชาติกาแฟแก้วละ99คือน้ำล้างเท้าดีๆนี่เอง
226 รสชาติ น้ำแข็งจะอัดเยอะไปไหน
230 รสชาติห่วยขั้นเทพ

In [30]:
token_widget = widgets.ToggleButtons(
    options=[num + 1 for num in range(num_clusters)],
    disabled=False,
    button_style='',
)

def on_token_widget_click(change):
    clear_output()
    display(token_widget)
    for index, value in result[result['predicted_label'] == change['new']]['tokenized_comment'].iteritems():
        print(index, value)

token_widget.observe(on_token_widget_click, names='index')
on_token_widget_click({'new' : 0})

ToggleButtons(options=(1, 2, 3, 4, 5, 6), value=1)

18 ['เหมือน', 'สาขา', 'รสชาติ', 'เหมือน', 'คน', 'ทำ', 'กาแฟ', 'รสชาติ', 'ลูกค้า', 'ทรู', 'สิทธิ์', 'ลด', 'พิเศษ']
20 ['รสชาติ', 'เหมือน', 'แก้ว', 'ชาเขียว', 'น้ำ', 'แข็ง', 'เย็น', 'แพง', 'คุณภาพ']
28 ['ซื้อ', 'หา', 'นั่ง', 'รสชาติ', 'น้ำ', 'แข็ง', 'ละลาย']
42 ['บ้าน', 'เหมือน', 'รสชาติ', 'ดูด', 'รู้', 'อารมณ์', 'ขม', 'หวาน', 'น้ำ', 'กิน', 'ขี้']
66 ['สั่ง', 'นึก', 'นม', 'รสชาติ', 'กา', 'แฟ', 'เรียน']
84 ['รสชาติ', 'แย่', 'จืด', 'แก้ว', 'โอเค']
103 ['รสชาติ']
130 ['รสชาติ', 'อร่อย', 'ไอ้', 'หวาน']
146 ['รสชาติ', 'กา', 'แฟ', 'เรื่อง', 'ขาย']
147 ['รสชาติ', 'กา', 'แฟ', 'จืด', 'อร่อย']
167 ['รสชาติ', 'งั้น']
168 ['สิทธิ์', 'รสชาติ', 'งั้น']
196 ['รสชาติ', 'อร่อย']
219 ['รสชาติ', 'กา', 'แฟ', 'แก้ว', 'น้ำ', 'ล้าง', 'ดี']
226 ['รสชาติ', 'น้ำ', 'แข็ง', 'อัด']
230 ['รสชาติ', 'ห่วย', 'แพง', 'สมราคา']
235 ['รสชาติ', 'บรรยากาศ', 'ร้าน']
236 ['ปรับปรุง', 'รสชาติ']
260 ['รสชาติ', 'กา', 'แฟ']
264 ['รู้สึก', 'แพง', 'รสชาติ', 'อ่า']
275 ['กาแฟ', 'รสชาติ', 'เหมือน']
326 ['รสชาติ', 'กา', 'แฟ', 'อ่ะ']
327